In [1]:
import urllib.request
import re
from bs4 import BeautifulSoup
from lxml import html
import pandas as pd
# import xmltojson
import json
import requests

In [2]:
def range_price(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    lowend = soup.find_all("span", {"class":"value", "itemprop":"lowprice", "content": True})
    highend = soup.find_all("span", {"class":"value", "itemprop":"highprice", "content": True})
    quantityList = []
    rangePrice = ""

    for i in lowend:
        rangePrice+=(i["content"])
        rangePrice+=" - "
        break

    for i in highend:
        rangePrice+=(i["content"])
        break

    return rangePrice


In [3]:
def scrape_handguns_helper(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
    url+= "=undefined&start=0&sz=100"
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    ammo = soup.find_all("div", class_="pdp-link")
    ammoPrice = soup.find_all("div",  class_="tile-body")
    names = []
    prices = []
    actual_prices = []
    for i in ammo:
        names.append(i.find("span").text)

    for i in ammoPrice:
        # if the product has a range of prices go here
        if '<span class="range">' in str(i):
            x = i.find("a", {"class": "link", "href":True})
            prices.append(x["href"])

        # if the product has a slashed price, go here
        else:
            x = (i.find("span", class_="sales"))
            prices.append(x.find("span", {"class":"value", "content": True}))


    # add prices in the prices list
    for i in prices:
        if "http" in i:
            rp = range_price(i)
            actual_prices.append(rp)

        else:
            htmlPriceTag = str(i)
            splitHtmlPriceTag = htmlPriceTag.split(" ")
            p =  splitHtmlPriceTag[2].strip('content="')
            actual_prices.append(str(p))
   
    try:
        productList = pd.DataFrame(
            {"name": names, "price": actual_prices}
        )
        return productList

    except:
        print("something went wrong")

In [4]:
# https://www.cheaperthandirt.com/9mm-luger-parabellum/ammunition/handgun-ammo/?https://www.cheaperthandirt.com/9mm-luger-parabellum/ammunition/handgun-ammo/=undefined&start=0&sz=36
handgunByCaliberLink = "https://www.cheaperthandirt.com/shop-by?cgid=78&searchBy=Caliber"
def scrape_handguns(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    sections = soup.find_all("ul",class_= "col-lg-11 right-list")

    # get caliber number set
    caliberNumbers = []
    x = soup.find_all('div', class_="refinements")
    for i in x:
        cn = i.find_all("div", {"id": True})
        for j in cn:
            caliberNumbers.append((j["id"]))

    dirtyLinks = []  
    
    for i in sections:
        dirtyLinks.append(i.find_all("a"))

    caliberNumberDF = pd.DataFrame(
        {"number": caliberNumbers, "section": dirtyLinks}
    )
    
    arrayOfDicts = []
    index = 0
    for i in caliberNumberDF["section"]:
        linkToCaliberList = []  # links
        productDict = {"sectionNumber": [], "name": [], "price": [], "link": []}
        for j in i:
            aLink = re.findall("(?P<url>https?://[^\s]+)", str(j))
            linkToCaliberList.append(str(aLink[0]).replace('">', ""))


        for k in linkToCaliberList:
            productDF = scrape_handguns_helper(k)
            aNameList = []
            aPriceList = []
            
            for j in productDF["name"]:
                aNameList.append(j)
            productDict["name"].append(aNameList)

            productDict["link"].append(k)
            

            for j in productDF["price"]:
                aPriceList.append(j)
            productDict["price"].append(aPriceList)
        
            
        
        productDict["sectionNumber"].append(caliberNumbers[index])
        arrayOfDicts.append(productDict)
        index += 1
    return arrayOfDicts


    

        



In [5]:
handgunDF = (scrape_handguns(handgunByCaliberLink))

In [6]:
for i in handgunDF:
    print(i)

{'sectionNumber': ['1'], 'name': [['Browning X-Point Defense 10mm Auto Ammunition 20 Rounds JHP 180 Grain', 'Hornady Critical Duty 10mm Auto Ammunition 20 Rounds FlexLock 175 Grains 91256', 'Magtech 10mm Auto Ammunition 50 Rounds FMJ 180 Grains', 'CCI Blazer 10mm Auto Ammunition 50 Rounds 200 Grain Full Metal Jacket 1050fps', 'Sellier & Bellot 10mm Ammunition 50 Rounds JHP 180 Grains', 'American Quality 10mm Auto Ammunition 250 Bulk Rounds FMJ 180 Grains N10180VP250']], 'price': [['29.89', '49.24', '49.89', '59.89', '59.89', '159.89']], 'link': ['https://www.cheaperthandirt.com/10mm-automatic/ammunition/handgun-ammo/?srule=price-low-to-high&amp;brand=10mmAutomatic']}
{'sectionNumber': ['2'], 'name': [['Winchester USA .25 ACP Ammunition 50 Rounds, FMJ, 50 Grains']], 'price': [['49.89']], 'link': ['https://www.cheaperthandirt.com/.25-acp-automatic/ammunition/handgun-ammo/?srule=price-low-to-high&amp;brand=.25ACP/Automatic']}
{'sectionNumber': ['3'], 'name': [['Buffalo Bore .32 H&R Magnum

In [19]:
from pathlib import Path
import os, shutil, glob
rel = "jsonFiles/cheaperThanDirt/ammo/handgun"
cwd = Path.cwd()
currentPath = str(Path.cwd())+""
aCount = 0
savePath = str(cwd).replace("pyScripts/cheaperThanDirt", rel)
index = 0
print(currentPath)
print(savePath)
# for i in handgunDF:
#     with open(os.path.join(savePath, f"CTDHandgunAmmo.json{index}"), "w") as fp:
#         json.dump(i, fp)
#         index += 1



/Users/chakaneshegog/Desktop/crispy-computing-machine/pyScripts/cheaperThanDirt
/Users/chakaneshegog/Desktop/crispy-computing-machine/jsonFiles/cheaperThanDirt/ammo/handgun/cheaperThanDirt
